In [17]:
import requests
import json, re
from bs4 import BeautifulSoup
import pymongo
from urllib.parse import urljoin
import pprint
from itertools import count

actor_url = "http://www.cine21.com/rank/person/content"
form_data['section'] = 'actor'
form_data['gender'] = 'all'
form_data['period_start'] = '2019-07'
actor_items = []

for page in count(1):
    form_data['page'] = page
    print(f'now on page {page}...')

    res = requests.post(url = actor_url, data = form_data)
    soup = BeautifulSoup(res.text, 'html.parser')

    names = soup.select('div.name')
    if len(names) == 0:
        break
    hit = soup.select('ul.num_info > li > strong')
    mov_list = soup.select('ul.mov_list')
    ranks = soup.select('.grade')

    actor = []
    ## this is novel (enumerate and index)
    for index, name in enumerate(names): 
        href = name.a['href']
        name = re.sub(r'\(\w*\)', '' ,name.text)

        print(f'scraping {name}\'s info..')

        movie_titles = mov_list[index].select('li a span') ## this is novel
        rank = ranks[index].text
        m_l = []
        for mt in movie_titles:
            m_l.append(mt.text)
        actor.append({
            'name' : name, 
            'href': href, 
            'hit' : int(hit[index].text.replace(',','')), ## this is novel
            'movie-list' : m_l,
            'rank' : int(rank),
            })

    
    for nh in actor:

        actor_item = {}
        actor_item['이름'] = nh['name']
        actor_item['흥행지수'] = nh['hit']
        actor_item['출연영화'] = nh['movie-list']
        actor_item['랭킹'] = nh['rank']

        url = urljoin(actor_url,nh['href'])
        res = requests.get(url)
        soup = BeautifulSoup(res.text, 'html.parser')

        for li in soup.select('.default_info li'):
            actor_item_key = li.select_one('span.tit').text
            # print(actor_item_key)
            actor_item_value = re.sub('<span.*?>.*?</span>|<.*?>','',str(li))
            regex = re.compile(r'[\n\r\t]')
            actor_item_value= regex.sub('',actor_item_value)
            actor_item[actor_item_key] = actor_item_value

        actor_items.append(actor_item)
print(f'Finish! - {len(actor_items)} people\'s data has appended')



scraping 이덕화's info..
scraping 소향's info..
now on page 159...
scraping 이예성's info..
scraping 주소정's info..
scraping 백승철's info..
scraping 윤계상's info..
scraping 이주예's info..
scraping 지's info..
scraping 노경희's info..
now on page 160...
scraping 노경희's info..
scraping 신일룡's info..
scraping 강봉성's info..
scraping 김혜자's info..
scraping 정연주's info..
scraping 황현빈's info..
scraping 고봉수's info..
now on page 161...
scraping 고봉수's info..
scraping 유정호's info..
scraping 유시민's info..
scraping 김성희's info..
scraping 강홍석's info..
scraping 오태경's info..
scraping 곽규석's info..
now on page 162...
scraping 이민영's info..
scraping 곽규석's info..
scraping 김승비's info..
scraping 송용태's info..
scraping 전혜영's info..
scraping 하준호's info..
scraping 성병숙's info..
now on page 163...
scraping 전혜영's info..
scraping 백인호's info..
scraping 김창숙's info..
scraping 민진웅's info..
scraping 서현철's info..
scraping 송경철's info..
scraping 이승호's info..
now on page 164...
scraping 조현영's info..
scraping 장진희's info..
scraping 주남정's info..
scraping

In [19]:
actor_list = [ actor for actor in actor_items if len(actor['이름']) < 10]


In [27]:
with open('./actor.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(actor_list, indent = 4, ensure_ascii = False))